# Problem 2


In [1]:
import numpy as np
import matplotlib.pyplot as plt

## 2C:
Fill in these functions to train your SVD

In [2]:
def grad_U(Ui, Yij, Vj, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    
    return eta*(reg*Ui-(Yij - np.dot(Ui,Vj))*Vj)

def grad_V(Vj, Yij, Ui, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    
    return eta*(reg*Vj - (Yij - np.dot(Ui,Vj))*Ui)

def get_err(U, V, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    num_entries = np.shape(Y)[0]
    error = reg/2*((np.linalg.norm(U))**2 + (np.linalg.norm(V))**2) 
    for entry in range(num_entries):
        i = Y[entry, 0]
        j = Y[entry, 1]
        Yij = Y[entry, 2]
        error  =  error + 1/2*(Yij - np.dot(U[i,:],V[j,:]))**2
    
    return error/len(Y)

def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=100):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV^T)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    
    #Initialize U and V
    U  = np.random.uniform(-.5,.5,(M+1,K))
    V  = np.random.uniform(-.5,.5,(N+1,K))
    
    zeroth_err = get_err(U, V, Y, reg)
    past_err = zeroth_err
    #Shuffle training data
    np.random.shuffle(Y)
    
    for epoch in range(max_epochs):
        print(epoch)
        num_entries = np.shape(Y)[0]
        for entry in range(num_entries):
            i = Y[entry, 0]
            j = Y[entry, 1]
            Yij = Y[entry, 2]

            Ui = U[i,:]
            Vj = V[j,:]

            delta_u = grad_U(Ui, Yij, Vj, reg, eta)
            delta_v = grad_V(Vj, Yij, Ui, reg, eta)

            U[i,:] = U[i,:] -  delta_u
            V[j,:] = V[j,:] -  delta_v
        
        #get the error for this epoch
        err =  get_err(U, V, Y, reg)
        nth_rel_err = abs(past_err - err)
        past_err = err
        
        if (epoch == 0):
            first_err = err
            first_rel_err = abs(zeroth_err-first_err)
            
        if (nth_rel_err/first_rel_err < .0001):
            break 
    
    return (U, V, err)

In [7]:
Y_train = np.loadtxt('./data/train.txt').astype(int)
Y_test = np.loadtxt('./data/test.txt').astype(int)

fix_indices = np.append(np.ones((90000,2)),np.zeros((90000,1)), axis = 1)
Y_train = (Y_train - fix_indices).astype(int)

fix_indices = np.append(np.ones((10000,2)),np.zeros((10000,1)), axis = 1)
Y_test = (Y_test - fix_indices).astype(int)


M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies
print("Factorizing with ", M, " users, ", N, " movies.")
K = 20

reg = 0.0
eta = 0.03 # learning rate
E_in = []
E_out = []

U,V, err = train_model(M, N, K, eta, reg, Y_train)

Factorizing with  942  users,  1681  movies.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


## 2E:
Run the cell below to get your graphs. This might take a long time to run, but it should take less than 2 hours. I would encourage you to validate your 2C is correct.

In [11]:
Y_train = np.loadtxt('./data/train.txt').astype(int)
Y_test = np.loadtxt('./data/test.txt').astype(int)

fix_indices = np.append(np.ones((90000,2)),np.zeros((90000,1)), axis = 1)
Y_train = (Y_train - fix_indices).astype(int)

fix_indices = np.append(np.ones((10000,2)),np.zeros((10000,1)), axis = 1)
Y_test = (Y_test - fix_indices).astype(int)


M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies
Ks = [20]

regs = [10**-4, 10**-3, 10**-2, 10**-1, 1]
eta = 0.03 # learning rate
E_ins = []
E_outs = []

# Use to compute Ein and Eout
for reg in regs:
    E_ins_for_lambda = []
    E_outs_for_lambda = []

    for k in Ks:
        print("Training model with M = %s, N = %s, k = %s, eta = %s, reg = %s"%(M, N, k, eta, reg))
        U,V, e_in = train_model(M, N, k, eta, reg, Y_train)
        E_ins_for_lambda.append(e_in)
        eout = get_err(U, V, Y_test)
        E_outs_for_lambda.append(eout)

    E_ins.append(E_ins_for_lambda)
    E_outs.append(E_outs_for_lambda)


# Plot values of E_in across k for each value of lambda
#for i in range(len(regs)):
#    plt.plot(Ks, E_ins[i], label='$E_{in}, \lambda=$'+str(regs[i]))
#plt.title('$E_{in}$ vs. K')
#plt.xlabel('K')
#plt.ylabel('Error')
#plt.legend()
#plt.savefig('2e_ein.png')	
#plt.clf()

# Plot values of E_out across k for each value of lambda
#for i in range(len(regs)):
#    plt.plot(Ks, E_outs[i], label='$E_{out}, \lambda=$'+str(regs[i]))
#plt.title('$E_{out}$ vs. K')
#plt.xlabel('K')
#plt.ylabel('Error')
#plt.legend()	
#plt.savefig('2e_eout.png')		


Training model with M = 942, N = 1681, k = 20, eta = 0.03, reg = 0.0001
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
Training model with M = 942, N = 1681, k = 20, eta = 0.03, reg = 0.001
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
Training model with M = 942, N = 1681, k = 20, eta = 0.03, reg = 0.01
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
Training model with M = 942, N = 1681, k = 20, eta = 0.03, reg = 0.1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
Training model with M = 942, N = 1681, k = 20, eta = 0.03, reg = 1
0
1
2
3
4
5
6
7
8
9


In [16]:
E_ins

[[0.19328769975288532],
 [0.19005906552782678],
 [0.19224985412589968],
 [0.2909500904404595],
 [0.9924451065250176]]

In [17]:
E_outs

[[0.7137142972996471],
 [0.6922800465169756],
 [0.60991970841457],
 [0.450200511489975],
 [0.9799553311146403]]

In [113]:
U,V, e_in = train_model(M, N, 20, .03, 10**-1, Y_train)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [114]:
U = np.transpose(U)
V = np.transpose(V)

In [121]:
means = []
for i in range(np.shape(V)[1]):
    means.append(np.transpose(np.mean(V,1)))
means = np.transpose(means)
V = V - means 

means = []
for i in range(np.shape(U)[1]):
    means.append(np.transpose(np.mean(V,1)))
means = np.transpose(means)
U = U - means


#Get SVD

A, S, B = np.linalg.svd(V)
V_tilde = np.matmul(np.transpose(A[:,0:2]),V)

A, S, B = np.linalg.svd(U)
U_tilde = np.matmul(np.transpose(A[:,0:2]),U)

In [129]:
np.shape(V)

(20, 1682)

In [78]:
a[:,0:2]

array([[1, 1],
       [2, 2],
       [3, 3],
       [4, 4]])

In [31]:
a[1]

array([2, 2, 2])

In [60]:
np.shape(a)[1]

3

In [91]:
a

array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3],
       [4, 4, 4]])

In [94]:
a[:,0:2]

array([[1, 1],
       [2, 2],
       [3, 3],
       [4, 4]])